# Sound Classifier

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from memory_profiler import memory_usage
import tensorflow as tf
from matplotlib import figure
import gc
from pathlib import Path

In [2]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K

import librosa
import librosa.display
import pylab
from keras_preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

**Creating Spectrograms**

In [3]:
#use the same function , by changing the directory for test set creating

def create_spectrogram_images(filename,name):
    plt.interactive(False)
    clip,sample_rate = librosa.load(filename,sr=None)
    fig = plt.figure(figsize=[0.74,0.74])
    
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    
    s = librosa.feature.melspectrogram(y=clip,sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(s,ref=np.max))
    
    filename = "C:/Users/reekithak/Deep Learning/Images/test/" + name + '.jpg'
    
    plt.savefig(filename,dpi=400,bbox_inches='tight',pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,s

In [4]:
def outside_fn():
    Data_dir = np.array(glob("C:/Users/reekithak/Deep Learning/English_split/Working/Test/Bad/*"))
    %load_ext memory_profiler
    %memit
    i =0 
    k=0
    for file in Data_dir[i:i+213]:
        #print(file)
        filename,name = file,file.split('/')[-1].split('.')[0]
        print(name,k)
        create_spectrogram_images(filename,name)
        k+=1

    gc.collect()

**Loading Spectrograms Images**

In [5]:
#images directory
train_dir = r"C:\Users\reekithak\Audio_Models\Data\Images\new_data\train"
test_dir = r"C:\Users\reekithak\Audio_Models\Data\Images\new_data\test"

In [6]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator = datagen.flow_from_directory(train_dir,
                                              batch_size=32,
                                              seed=42,
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=(64,64)
)

Found 1662 images belonging to 2 classes.


In [8]:
test_generator = datagen.flow_from_directory(test_dir,
                                              batch_size=32,
                                              seed=42,
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=(64,64)
)

Found 554 images belonging to 2 classes.


**Architecture Building**

In [9]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [10]:
model.compile(optimizers.RMSprop(lr=0.005,decay=1e-6),
             loss="categorical_crossentropy",
             metrics=["acc"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        3

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 32
def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        #horizontal_flip = True, 
        validation_split = 0.25,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width), 
        classes = ('bad','good'),
        batch_size = batch_size,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width), 
        classes = ('bad','good'),
        batch_size = batch_size,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [12]:
def plot_history(history, yrange):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    
    plt.show()

In [13]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
EARLY_STOP_PATIENCE = 3
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = 'C:\\Users\\reekithak\\Audio_Models\\5.resnet\\new_res50.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [14]:
train_step_size = train_generator.n//train_generator.batch_size
test_step_size = test_generator.n//test_generator.batch_size

In [15]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=train_step_size,
                    validation_data=test_generator,
                    validation_steps=test_step_size,
                    epochs=500,
                    callbacks=[cb_checkpointer, cb_early_stopper]
                
)

c:\users\reekithak\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
51/51 [==============================] - 31s 583ms/step - loss: 24.2218 - acc: 0.4918 - val_loss: 0.6929 - val_acc: 0.5184
Epoch 2/500
51/51 [==============================] - 29s 563ms/step - loss: 0.6944 - acc: 0.5038 - val_loss: 0.6925 - val_acc: 0.5184
Epoch 3/500
51/51 [==============================] - 29s 563ms/step - loss: 0.6939 - acc: 0.5013 - val_loss: 0.6929 - val_acc: 0.5184
Epoch 4/500
51/51 [==============================] - 122s 2s/step - loss: 0.6945 - acc: 0.4932 - val_loss: 0.6940 - val_acc: 0.5184
Epoch 5/500
51/51 [==============================] - 134s 3s/step - loss: 0.6926 - acc: 0.5345 - val_loss: 0.6934 - val_acc: 0.4853


In [16]:
model.save("new_naive.h5")

**Building the default converter and tester**